In [1]:
import numpy as np
from bokeh.io import show

# Small Example

In [ ]:
from multiscale_BM_utils import plot_multiscale_BM

In [ ]:
d_matrix = np.loadtxt('output/test_1_edges', delimiter=',')
d_matrix

In [ ]:
distance_from_landmarks = np.loadtxt('output/test_1_distance_of_points_in_order_from_landmarks', delimiter=',')
distance_from_landmarks

In [ ]:
color_of_landmarks = np.loadtxt('output/test_1_coloration_in_order_from_landmarks', delimiter=',')
color_of_landmarks

In [ ]:
multiscale_BM = plot_multiscale_BM(d_matrix, 
                          distance_from_landmarks,
                          color_of_landmarks,)

In [ ]:
show(multiscale_BM)

# Knots Example

In [2]:
from multiscale_BM_utils import plot_sampled_multiscale_BM

In [3]:
d_matrix = np.loadtxt('output/jones13n_20_edges', delimiter=',')
d_matrix.shape

(74, 74)

In [4]:
distance_from_landmarks = np.loadtxt('output/jones13n_20_distance_of_points_in_order_from_landmarks', delimiter=',')
distance_from_landmarks.shape

(74, 12472)

In [5]:
color_of_landmarks = np.loadtxt('output/jones13n_20_coloration_in_order_from_landmarks', delimiter=',')
color_of_landmarks.shape

(74, 12472)

In [ ]:
sampled_multiscale_BM = plot_sampled_multiscale_BM(d_matrix, 
                                                   distance_from_landmarks,
                                                   color_of_landmarks,
                                                   num_of_points = 1000,
                                                   exponential_scaling=True)

In [ ]:
show(sampled_multiscale_BM)

# export png

In [6]:
from multiscale_BM_utils import save_graph_to_png

In [11]:
from selenium import webdriver

In [15]:
driver = webdriver.Chrome('/Users/davide/GitHub/multiscaleBM/chromedriver') 

In [16]:
epsilon_list = [10, 20, 50]

save_graph_to_png(epsilon_list, driver, 
                  d_matrix,
                  distance_from_landmarks,
                  color_of_landmarks,
                  plot_width = 700,
                  plot_height = 700,
                  filename = 'img/plot', 
                  add_colorbar = False,
                  exponential_scaling=True)